In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, GlobalAveragePooling2D
import os
import cv2
import pickle
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Model


from imblearn.under_sampling import RandomUnderSampler
from keras.utils.np_utils import to_categorical
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
dataPath = "../../data/OCT/OCT2017/"

In [3]:
labels_available = os.listdir(os.path.join(dataPath, "train"))
print("Total Number of Classes Detected :",len(labels_available))

labels_list = ['NORMAL',"CNV","DME","DRUSEN"]

Total Number of Classes Detected : 4


In [4]:
images=[]
y_trn=[]
for x in labels_list:
    xPath = os.path.join(dataPath, "train", x)
    myPicList = os.listdir(xPath)
    for y in myPicList:
        images.append(cv2.imread(os.path.join(xPath, y)))
        y_trn.append(labels_list.index(x))
    print(x ,end=" ")

y_train = to_categorical(y_trn,len(labels_list))

NORMAL CNV DME DRUSEN 

In [5]:
x_val=[]
y_val=[]
for x in labels_list:
    xPath = os.path.join(dataPath, "val", x)
    myPicList = os.listdir(xPath)
    for y in myPicList:
        x_val.append(cv2.imread(os.path.join(xPath, y)))
        y_val.append(labels_list.index(x))
    print(x ,end=" ")

y_validation = to_categorical(y_val,len(labels_list))

NORMAL CNV DME DRUSEN 

In [6]:
x_tst=[]
y_tst=[]
for x in labels_list:
    xPath = os.path.join(dataPath, "test", x)
    myPicList = os.listdir(xPath)
    for y in myPicList:
        x_tst.append(cv2.imread(os.path.join(xPath, y)))
        y_tst.append(labels_list.index(x))
    print(x ,end=" ")

NORMAL CNV DME DRUSEN 

In [7]:
def resizeIm(im, size):
    if im.shape[2] == 1:
        im = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
    return cv2.resize(im, size)

def resizeIms(x, size):
    return np.array(list(map(lambda im: resizeIm(im, size), x)))

In [8]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self,fraction, model):
        super(CustomCallback,self).__init__()
        self.fraction = fraction
        self.train_a = [];
        self.val_a =[];
        self.logPath = os.path.join(model, "log.txt")

        if not os.path.isdir(model):
            os.mkdir(model)

        with open(self.logPath,'w') as f:
            f.write('Starting of logging..\n')

        self.fig = plt.figure(figsize=(4,3))
        self.ax = plt.subplot(1,1,1)
        plt.ion()

    def on_train_begin(self,logs=None):
        self.fig.show()
        self.fig.canvas.draw()

    def on_train_end(self,logs=None):
        with open(self.logPath,'a') as f:
              f.write('End of logging..\n')
    
    def on_epoch_begin(self,epoch,logs=None):
        lr= tf.keras.backend.get_value(self.model.optimizer.lr)
        lr *= self.fraction
        tf.keras.backend.set_value(self.model.optimizer.lr,lr)
        with open(self.logPath,'a') as f:
            f.write('At epoch {:02d}, learning rate changed to {:.4f}\n'.format(epoch,lr))
    
    def on_epoch_end(self,epoch,logs=None):
        val_acc = logs.get('val_accuracy')
        train_acc = logs.get('accuracy')
        self.train_a.append(train_acc)
        self.val_a.append(val_acc)
        with open(self.logPath,'a') as f:
            f.write('At epoch {:02d}, training accuracy: {:.3f}, validation accuracy: {:.3f}\n'.format(epoch,train_acc,val_acc))
        self.ax.clear()
        self.ax.plot(range(1+epoch),self.train_a,label="Training")
        self.ax.plot(range(1+epoch),self.val_a,label="Validation")
        self.ax.set_xlabel('Epochs')
        self.ax.set_ylabel('Accuracy')
        self.ax.legend()
        self.fig.canvas.draw()
        self.fig.show()

## Load models

In [14]:
# xception base model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
xceptionNetModel = Xception(weights='imagenet')

# opticnet base model
opticNetModel = tf.keras.models.load_model('../Optic_net-4_classes-Kermany2018.hdf5')

# resnet base model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

resNetModel = ResNet50(weights='imagenet')

In [15]:
def emptyModelGenerator(model):
    model = model.lower()
    if model == "resnet":
        x = resNetModel.output
        predictions = Dense(len(labels_list), activation='softmax')(x)
        newModel = Model(inputs=resNetModel.input, outputs=predictions)
        size = (224, 224)
    elif model == "xception":
        x = xceptionNetModel.output
        predictions = Dense(len(labels_list), activation='softmax')(x)
        newModel = Model(inputs=xceptionNetModel.input, outputs=predictions)
        size = (299, 299)
    elif model == "opticnet":
        newModel = tf.keras.models.load_model('../Optic_net-4_classes-Kermany2018.hdf5')
        size = (224, 224)
    return newModel, size


def testPredict(model, size):
    X_test = resizeIms(x_tst, size)
    X_test = np.array(X_test)
    Y_test = np.array(y_tst)
    
    prediction = model.predict(X_test)
    preds = np.argmax(prediction, axis=1)
    acc = sum(preds == Y_test) / len(Y_test)
    print(f'Test acc for {model}: {acc:.4f}')
    

nOptic = emptyModelGenerator("opticnet")

In [16]:
optim = Adam(learning_rate=0.001)
epochs = 30
batch_size = 50

In [ ]:
from sklearn.model_selection import train_test_split

maxTrain = len(y_train)
for p in [0.1, 0.25, 0.4, 0.5, 0.6, 0.75, 0.9]:
    #X_trn, X_tst, y_trn, y_tst
    X_t, _, y_t, _ = train_test_split(images, y_train, test_size=1-p, random_state=123)
    for net in ["opticnet", "xception", "resnet"]:
        print(f"Training {net} for {p}% of train size (aka {len(X_t)} images)...")
        model, size = emptyModelGenerator(net)
        X_trn = resizeIms(X_t, size)
        X_val = resizeIms(x_val, size)
        model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
        hist = model.fit(X_trn, y_t, epochs=epochs, validation_data = (X_val, y_validation), batch_size=batch_size,
                    shuffle=True, max_queue_size=20,
                    use_multiprocessing=True, workers=5, 
                    callbacks=[CustomCallback(fraction=0.9, model=net)])
        model.save(f"../{net}/{net}_{epochs}epochs_{p*maxTrain} images")
        testPredict(model, size)
        print("Done!")

Training opticnet for 0.1% of train size (aka 8348 images)...


<ipython-input-8-797fd00c81c1>:20: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 1/30
167/167 [==============================] - 216s 1s/step - loss: 0.9171 - accuracy: 0.7730 - val_loss: 0.0495 - val_accuracy: 1.0000
Epoch 2/30


<ipython-input-8-797fd00c81c1>:48: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


167/167 [==============================] - 196s 1s/step - loss: 0.1521 - accuracy: 0.9507 - val_loss: 0.1273 - val_accuracy: 0.9688
Epoch 3/30
167/167 [==============================] - 196s 1s/step - loss: 0.1070 - accuracy: 0.9658 - val_loss: 0.0051 - val_accuracy: 1.0000
Epoch 4/30
167/167 [==============================] - 197s 1s/step - loss: 0.0853 - accuracy: 0.9729 - val_loss: 0.0405 - val_accuracy: 1.0000
Epoch 5/30
167/167 [==============================] - 197s 1s/step - loss: 0.0649 - accuracy: 0.9777 - val_loss: 0.0556 - val_accuracy: 1.0000
Epoch 6/30
167/167 [==============================] - 197s 1s/step - loss: 0.0422 - accuracy: 0.9885 - val_loss: 0.0325 - val_accuracy: 1.0000
Epoch 7/30
167/167 [==============================] - 197s 1s/step - loss: 0.0316 - accuracy: 0.9887 - val_loss: 0.0192 - val_accuracy: 1.0000
Epoch 8/30
167/167 [==============================] - 196s 1s/step - loss: 0.0160 - accuracy: 0.9954 - val_loss: 3.8572e-04 - val_accuracy: 1.0000
Epoch 

<ipython-input-8-797fd00c81c1>:20: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 1/30
167/167 [==============================] - 134s 772ms/step - loss: 1.3342 - accuracy: 0.7390 - val_loss: 1.2893 - val_accuracy: 0.9688
Epoch 2/30


<ipython-input-8-797fd00c81c1>:48: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


167/167 [==============================] - 131s 781ms/step - loss: 1.2713 - accuracy: 0.9248 - val_loss: 1.2713 - val_accuracy: 0.9688
Epoch 3/30
167/167 [==============================] - 130s 781ms/step - loss: 1.2491 - accuracy: 0.9362 - val_loss: 1.2555 - val_accuracy: 1.0000
Epoch 4/30
167/167 [==============================] - 130s 780ms/step - loss: 1.2335 - accuracy: 0.9518 - val_loss: 1.2460 - val_accuracy: 1.0000
Epoch 5/30
167/167 [==============================] - 130s 780ms/step - loss: 1.2182 - accuracy: 0.9623 - val_loss: 1.2380 - val_accuracy: 1.0000
Epoch 6/30
167/167 [==============================] - 130s 780ms/step - loss: 1.2103 - accuracy: 0.9561 - val_loss: 1.2345 - val_accuracy: 0.9688
Epoch 7/30
167/167 [==============================] - 129s 775ms/step - loss: 1.2008 - accuracy: 0.9644 - val_loss: 1.2261 - val_accuracy: 1.0000
Epoch 8/30
167/167 [==============================] - 131s 781ms/step - loss: 1.1948 - accuracy: 0.9642 - val_loss: 1.2268 - val_accura